In [1]:
import os
import copy
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
import sys
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import ParameterGrid
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
dataset_path = './data/wikiart_sample/'
dataset = ImageFolder(root=dataset_path, transform=transform)

In [4]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [5]:
class CustomResNet50(nn.Module):
    def __init__(self, dropout_rate=0.0, num_classes=27):
        super(CustomResNet50, self).__init__()
        self.resnet50_model = models.resnet50(pretrained=True)
        num_ftrs = self.resnet50_model.fc.in_features
        self.resnet50_model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.resnet50_model(x)

In [6]:
param_grid = {
    'batch_size': [16, 32],
    'lr': [0.0001, 0.001],
    'dropout_rate': [0.1, 0.5],
    'optimizer': ['Adam', 'RMSprop'],
    'weight_decay': [0, 0.01]
}

In [7]:
# Create Parameter grid
param_list = list(ParameterGrid(param_grid))

best_model = None
best_params = None
best_accuracy = 0.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
print(f"Using device: {device}")

Using device: cuda


In [9]:
performance_dict = {}
for params in param_list:
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)

    model = CustomResNet50(dropout_rate=params['dropout_rate'])
    model.to(device)

    if params['optimizer'] == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    elif params['optimizer'] == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])

    criterion = nn.CrossEntropyLoss()

    num_epochs = 1
    current_best_accuracy = 0.0  # Reset for each hyperparameter set

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train
        print(f"\n Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training accuracy: {train_accuracy}")

        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        actual_accuracy = 100 * correct_val / total_val
        print(f'Validation accuracy after epoch {epoch+1}: {actual_accuracy:.2f}%')

        if actual_accuracy > current_best_accuracy:
            current_best_accuracy = actual_accuracy

        if actual_accuracy > best_accuracy:
            best_model = copy.deepcopy(model)
            best_params = params
            best_accuracy = actual_accuracy

    # After all epochs for this hyperparameter set
    key = str(params)
    performance_dict[key] = current_best_accuracy
    print(f"Best validation accuracy for this set of hyperparameters ({params}): {current_best_accuracy:.2f}%")
    print(f"Overall best validation accuracy so far: {best_accuracy:.2f}% with hyperparameters {best_params}")

# After all hyperparameter sets
print("\nOverall best hyperparameters: ", best_params)
print(f"Overall best validation accuracy: {best_accuracy:.2f}%")

print("\nPerformance dictionary:")
print(performance_dict)

# Save the best model
torch.save(best_model, "wiki_TL_resnet_model_hyp.pth")


 Epoch 1/1, Loss: 2.2919089508056643, Training accuracy: 32.02303455182774
Validation accuracy after epoch 1: 42.34%
Best validation accuracy for this set of hyperparameters ({'batch_size': 16, 'dropout_rate': 0.1, 'lr': 0.0001, 'optimizer': 'Adam', 'weight_decay': 0}): 42.34%
Overall best validation accuracy so far: 42.34% with hyperparameters {'batch_size': 16, 'dropout_rate': 0.1, 'lr': 0.0001, 'optimizer': 'Adam', 'weight_decay': 0}

 Epoch 1/1, Loss: 2.286488968372345, Training accuracy: 31.29694541812719
Validation accuracy after epoch 1: 40.84%
Best validation accuracy for this set of hyperparameters ({'batch_size': 16, 'dropout_rate': 0.1, 'lr': 0.0001, 'optimizer': 'Adam', 'weight_decay': 0.01}): 40.84%
Overall best validation accuracy so far: 42.34% with hyperparameters {'batch_size': 16, 'dropout_rate': 0.1, 'lr': 0.0001, 'optimizer': 'Adam', 'weight_decay': 0}

 Epoch 1/1, Loss: 2.3336294145584104, Training accuracy: 29.118678017025537
Validation accuracy after epoch 1: 41

In [10]:
# performance_dict
import pandas as pd
perf_summary = pd.DataFrame.from_dict(performance_dict, orient ='index')
perf_summary.reset_index(inplace = True)
perf_summary.rename(columns = {'index':'parameters',0:'accuracy'}, inplace = True)
perf_summary.sort_values(by = 'accuracy')
perf_summary.to_csv("performance_summary.csv", index = False)

In [11]:
# torch.save(model, "wiki_TL_resnet_model.pth")
# print(f"Best model saved with accuracy: {best_accuracy:.2f}%")

In [12]:
# Calculate the most frequent label in the training set
label_counts = Counter([label for _, label in train_dataset])
most_common_label, frequency = label_counts.most_common(1)[0]

# Calculate the expected accuracy if no model (always predicting the most frequent label)
baseline_accuracy = (frequency / len(train_dataset)) * 100
print(f"Expected accuracy if no model: {baseline_accuracy:.2f}%")

Expected accuracy if no model: 4.66%
